In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn import preprocessing
import random

### Importing OTU table, normalization (change to relative abundances), set training and testing groups

In [51]:
# Import data into a pandas dataframe
otu_table = pd.read_csv('EGAD00001003453.otu_table.99.denovo', sep='\t', header=0, index_col=0)

#Transpose OTU table so that samples are rows and OTUs are columns
otu_table = otu_table.T

#Convert to relative abundances
#Normalize each person to 1 (change to relative abundances)
otu_table_normalized = otu_table.div(otu_table.sum(axis=1), axis=0)
print(otu_table_normalized)

OTU_ID  denovo7709  denovo5395  denovo11322  denovo44859  denovo44858  \
G37960    0.000000         0.0          0.0          0.0          0.0   
G37961    0.000000         0.0          0.0          0.0          0.0   
G37962    0.000000         0.0          0.0          0.0          0.0   
G37963    0.000000         0.0          0.0          0.0          0.0   
G37964    0.000000         0.0          0.0          0.0          0.0   
G37965    0.000000         0.0          0.0          0.0          0.0   
G37966    0.000000         0.0          0.0          0.0          0.0   
G37967    0.000000         0.0          0.0          0.0          0.0   
G37968    0.000000         0.0          0.0          0.0          0.0   
G37969    0.000000         0.0          0.0          0.0          0.0   
G37970    0.000000         0.0          0.0          0.0          0.0   
G37971    0.000000         0.0          0.0          0.0          0.0   
G37972    0.000000         0.0          0.0        

In [52]:
#Make training and testing sets (50/50 for now)

#Scramble existing table with hard-coded random seed
otu_table_normalized = otu_table_normalized.sample(frac=1, random_state=1)

#Make first half training and second half testing
total_people = otu_table_normalized.shape[0]
print(total_people)
midway = total_people/2
training_table = otu_table_normalized.iloc[0:int(midway)]
testing_table = otu_table_normalized.iloc[int(midway):]


1010


### Set conditions to test:

In [53]:
N = [5, 10, 50, 100, 250]
weight_setup = ['even', 'uneven_100x', 'uneven_10x', 'uneven_graduated']#, 'uneven_bins']
num_bins = 3

#Number of times to run each condition (N and weight setup)
num_repeats = 5

### Functions

In [54]:
def calculate_weights(n,weight):
    if weight == 'even':
        return [1/n]*n
    elif weight == 'uneven_100x':
        x = 1/(100 + (n-1))
        return [100*x] + [x]*(n-1)
    elif weight == 'uneven_10x':
        x = 1/(10 + (n-1))
        return [10*x] + [x]*(n-1)
    elif weight == 'uneven_graduated':
        decreasing_list = list(range(n,0,-1))
        return [x/sum(decreasing_list) for x in decreasing_list]
    #elif weight == 'uneven_bins':
        #n/

    

### Master loop

In [88]:
state = [[0]*len(N) for i in range(num_repeats)]
df_list = []

for i in range(num_repeats):
    for j, n in enumerate(N):
        #Keep track of random seeds used to generate N random samples from OTU table
        state[i][j] = random.getstate()
        #Generate random sample of size n
        random_sample = training_table.sample(n=n, replace=True)
        
        for m, weight in enumerate(weight_setup):
            #Set up actual weights
            weight_list = calculate_weights(n,weight)
            
            #Calculate weighted OTU table
            #weighted_table = [random_sample.iloc[i]*weight_list[i] for i in range(n)]
            for k in range(n):
                random_sample.iloc[k] = random_sample.iloc[k]*weight_list[k]
                
            #Calculate weighted mean
            indiv_series = random_sample.mean(axis=0)
            
            #Store data + info/metadata as pandas Series
            sample_name = str(n) + "_" + weight + "_" + str(i)
            sample_number = i*len(N) + j*len(weight_setup) + m
            indiv_series.loc['sample_name'] = sample_name
            indiv_series.loc['N'] = n
            indiv_series.loc['weight_setup'] = weight
            indiv_series.loc['weight_vector'] = weight_list
            indiv_series.loc['random_seed'] = random.getstate()
            
            
            indiv_df = pd.DataFrame(indiv_series, columns=[str(sample_number)])
            #print(indiv_df)
            df_list.append(indiv_df)
            
final_matrix = pd.concat(df_list, axis=1)
print(final_matrix.tail())

                                                               0  \
OTU_ID                                                             
sample_name                                             5_even_0   
N                                                              5   
weight_setup                                                even   
weight_vector                          [0.2, 0.2, 0.2, 0.2, 0.2]   
random_seed    (3, (2147483648, 2561330682, 2716738939, 30638...   

                                                               1  \
OTU_ID                                                             
sample_name                                      5_uneven_100x_0   
N                                                              5   
weight_setup                                         uneven_100x   
weight_vector  [0.9615384615384616, 0.009615384615384616, 0.0...   
random_seed    (3, (2147483648, 2561330682, 2716738939, 30638...   

                                              

In [89]:
final_matrix

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
OTU_ID,,,,,,,,,,,,,,,,,,,,,
denovo7709,4.40533e-07,4.2359e-09,3.02564e-10,2.01709e-11,0,0,0,0,2.87003e-08,1.92619e-10,...,0,0,8.2205e-09,4.1309e-11,3.78982e-13,2.77762e-15,4.94542e-09,1.41703e-11,5.47115e-14,2.69599e-16
denovo5395,0,0,0,0,3.37194e-07,3.09352e-09,1.62817e-10,1.85928e-11,4.06391e-08,2.72746e-10,...,3.62704e-12,8.61517e-14,2.99192e-08,1.50348e-10,1.37934e-12,1.49662e-14,7.33152e-09,2.10072e-11,8.11089e-14,3.10467e-16
denovo11322,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
denovo44859,0,0,0,0,0,0,0,0,1.35953e-08,9.12434e-11,...,0,0,1.04949e-09,5.27384e-12,4.83839e-14,6.32343e-16,9.14808e-10,2.62123e-12,1.01206e-14,4.00481e-17
denovo44858,0,0,0,0,0,0,0,0,1.10389e-08,7.40866e-11,...,2.00979e-13,5.98997e-15,4.41702e-10,2.21961e-12,2.03634e-14,1.08873e-16,1.17464e-09,3.36574e-12,1.29951e-14,6.68489e-17
denovo41472,0,0,0,0,0,0,0,0,4.38553e-09,2.94331e-11,...,4.78581e-13,8.25787e-15,0,0,0,0,6.65302e-10,1.90631e-12,7.36027e-15,1.98746e-17
denovo41473,0,0,0,0,0,0,0,0,8.01659e-09,5.38027e-11,...,6.5102e-13,1.22545e-14,3.87687e-09,1.94818e-11,1.78732e-13,2.86679e-15,1.45688e-09,4.17444e-12,1.61175e-14,4.8454e-17
denovo41470,0,0,0,0,0,0,0,0,4.10084e-09,2.75224e-11,...,4.4536e-13,9.08185e-15,0,0,0,0,8.25019e-10,2.36395e-12,9.12723e-15,4.91533e-17
denovo41471,0,0,0,0,0,0,0,0,1.01668e-08,6.82335e-11,...,0,0,2.13857e-09,1.07466e-11,9.85923e-14,1.281e-15,3.4721e-10,9.94871e-13,3.8412e-15,1.42171e-17


In [92]:
final_matrix.drop(final_matrix.index[0])

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
OTU_ID,,,,,,,,,,,,,,,,,,,,,
denovo5395,0,0,0,0,3.37194e-07,3.09352e-09,1.62817e-10,1.85928e-11,4.06391e-08,2.72746e-10,...,3.62704e-12,8.61517e-14,2.99192e-08,1.50348e-10,1.37934e-12,1.49662e-14,7.33152e-09,2.10072e-11,8.11089e-14,3.10467e-16
denovo11322,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
denovo44859,0,0,0,0,0,0,0,0,1.35953e-08,9.12434e-11,...,0,0,1.04949e-09,5.27384e-12,4.83839e-14,6.32343e-16,9.14808e-10,2.62123e-12,1.01206e-14,4.00481e-17
denovo44858,0,0,0,0,0,0,0,0,1.10389e-08,7.40866e-11,...,2.00979e-13,5.98997e-15,4.41702e-10,2.21961e-12,2.03634e-14,1.08873e-16,1.17464e-09,3.36574e-12,1.29951e-14,6.68489e-17
denovo41472,0,0,0,0,0,0,0,0,4.38553e-09,2.94331e-11,...,4.78581e-13,8.25787e-15,0,0,0,0,6.65302e-10,1.90631e-12,7.36027e-15,1.98746e-17
denovo41473,0,0,0,0,0,0,0,0,8.01659e-09,5.38027e-11,...,6.5102e-13,1.22545e-14,3.87687e-09,1.94818e-11,1.78732e-13,2.86679e-15,1.45688e-09,4.17444e-12,1.61175e-14,4.8454e-17
denovo41470,0,0,0,0,0,0,0,0,4.10084e-09,2.75224e-11,...,4.4536e-13,9.08185e-15,0,0,0,0,8.25019e-10,2.36395e-12,9.12723e-15,4.91533e-17
denovo41471,0,0,0,0,0,0,0,0,1.01668e-08,6.82335e-11,...,0,0,2.13857e-09,1.07466e-11,9.85923e-14,1.281e-15,3.4721e-10,9.94871e-13,3.8412e-15,1.42171e-17
denovo41476,0,0,0,0,0,0,0,0,0,0,...,1.0832e-12,2.63367e-14,2.84038e-09,1.42733e-11,1.30948e-13,1.99064e-15,0,0,0,0


In [66]:
tmp = random_sample['denovo7709']
print(tmp.tail())
tmp.loc['hello'] = 'hi'
tmp.tail()

G43433    0.000000e+00
G38024    1.517569e-16
G38083    0.000000e+00
G43980    0.000000e+00
Mean      1.323793e-15
Name: denovo7709, dtype: float64


G38024    1.51757e-16
G38083              0
G43980              0
Mean      1.32379e-15
hello              hi
Name: denovo7709, dtype: object

In [ ]:
#Choose number of samples
num_samples = 5

#Choose num_samples random people from the study, with replacement
random_sample = training_table.sample(n=num_samples, replace=True)
print(random_sample)

In [ ]:
#Make into numpy array
numpy_array = random_sample.values
print(numpy_array)

In [ ]:
random_sample * pd.Series([0.8, 0.1, 0.1, 0.1, 0.1])

In [ ]:
#Take column means (mean OTU relative abundance across all sampled people)
#Even mixing
even_mean = numpy_array_normalized.mean(axis=0)
print(even_mean)


In [ ]:
#Uneven mixing: Repeat first person in sample num_samples*5 times, and calculate the new mean (this is in essence a weighted mean)
repeats = np.tile(numpy_array_normalized[0], (num_samples*5,1))
print(repeats)

uneven_array = np.vstack((numpy_array_normalized,repeats))
uneven_mean = uneven_array.mean(axis=0)
print(uneven_mean)

In [ ]:
#Put even and uneven means back into one dataframe
mean_df = pd.DataFrame(even_mean, columns=['even'], index=random_sample.columns)
mean_df['uneven'] = uneven_mean


In [ ]:
print(mean_df)

In [ ]:
nonzero_mean = mean_df.loc[(mean_df!=0).any(axis=1)]
print (nonzero_mean.shape)

In [ ]:
start = time.time()
plt.hist(nonzero_mean)
print (time.time()-start)

In [ ]:
%matplotlib inline
otu_table.sum().apply(np.log10).plot(kind='hist', bins=20)

In [ ]:
sum(otu_table.sum() <= 100)